In [10]:
# SageMath Script to Generate Matroid Example Files
#
# INSTRUCTIONS:
# 1. Run this script in a SageMath environment (e.g., Sage REPL, SageCell, or a Jupyter Notebook with a Sage kernel).
# 2. The script will automatically create several .py files in the current directory
#    (e.g., u_2_3.py, u_2_4.py, fano_matroid.py).
# 3. These generated files contain the matroid data in the standardized dictionary
#    format, ready to be used by other AI tools.

from sage.all import *
import sys
sys.path.append('/home/nnutannep/Github/simple-semi-simple/custom_matroid_functions')

from core import *
from polynomials import *

# --- Part 1: Core Computational Function ---

def get_poincare_polynomial(M):
    """
    Computes the Poincare polynomial H_M(t) for a given SageMath matroid M.

    This function uses the relationship between the Poincare polynomial and the
    matroid's characteristic polynomial, chi_M(x):
        H_M(t) = (-1)^r * t^r * chi_M(-1/t)
    where r is the rank of the matroid M.

    Args:
        M (sage.matroids.matroid.Matroid): A SageMath matroid object.

    Returns:
        (string): A string representation of the Poincare polynomial, e.g., "1 + 3*t".
    """
    # Define the polynomial ring for t
    R = PolynomialRing(ZZ, 't')
    t = R.gen()

    # Get the characteristic polynomial from Sage
    chi_M = M.characteristic_polynomial()
    r = M.rank()

    # Apply the conversion formula
    # The `subs()` method substitutes -1/t for the generator of the characteristic polynomial's ring
    poincare_poly = ((-1)^r) * (t^r) * chi_M.subs({chi_M.gen(): -1/t})

    # Simplify the resulting expression to get a clean polynomial
    simplified_poly = poincare_poly.simplify_full()

    # Convert the symbolic polynomial to a user-friendly string format
    # e.g., "t^2 + 3*t + 1" becomes "t**2 + 3*t + 1"
    return str(simplified_poly).replace("^", "**")


# --- Part 2: File Generation Function ---

def create_example_file(matroid, filename_prefix):
    """
    Generates a .py file for a given matroid in the standardized dictionary format.

    Args:
        matroid (sage.matroids.matroid.Matroid): The matroid to process.
        filename_prefix (string): The base name for the output file (e.g., "u_2_4").
    """
    print(f"Processing {matroid}...")

    # Get the lattice of flats. In Sage, this is the geometric lattice.
    lattice = matroid.lattice_of_flats()
    flats = list(lattice) # Convert to a list to have a stable order

    # Start building the dictionary structure
    example_data = {
        "name": str(matroid),
        "ground_set": str(matroid.groundset()),
        "rank": matroid.rank(),
        "lattice_of_flats": {}
    }

    # Process each flat in the lattice
    for i, F_elements in enumerate(flats):
        # Create a unique, human-readable key for the flat
        # e.g., F_empty, F_1, F_12
        flat_label = "F_" + "".join(map(str, sorted(list(F_elements)))) if F_elements else "F_empty"

        # Get the contraction of the original matroid at the current flat
        # M_F = M.contract(F_elements) does not work as intended for flats.
        # The correct operation is M/F, which corresponds to the interval [F, 1] in the lattice.
        # In Sage, this is done by creating a new matroid from the flats containing F.
        upper_flats = [up_F for up_F in flats if F_elements.issubset(up_F)]
        
        # We need to relabel the elements of the contracted matroid to create a valid new matroid.
        # However, a simpler way is to get the sub-lattice from the current flat to the top.
        sub_lattice = lattice.sublattice(lattice.subposet(elements=upper_flats))
        
        # Create the contracted matroid from this sub-lattice
        contracted_matroid = Matroid(sub_lattice)

        # Compute the Poincare polynomial for the contraction M/F
        poly_str = get_poincare_polynomial(contracted_matroid)

        # Add the flat's data to our dictionary
        example_data["lattice_of_flats"][flat_label] = {
            "elements": str(F_elements),
            "rank": matroid.rank_of_flats([F_elements])[0],
            "poincare_polynomial": poly_str,
        }
        print(f"  - Flat {flat_label}: H_{matroid.name()}/{flat_label}(t) = {poly_str}")


    # --- Format the final output string for the .py file ---
    
    # Header for the file
    file_content = f"#\n# Example data for the {matroid.name()}\n#\n"
    file_content += "# This file was automatically generated by a SageMath script.\n\n"
    file_content += "example = {\n"
    file_content += f"    \"name\": \"{example_data['name']}\",\n"
    file_content += f"    \"ground_set\": {example_data['ground_set']},\n"
    file_content += f"    \"rank\": {example_data['rank']},\n"
    file_content += f"    \"lattice_of_flats\": {{\n"

    # Add each flat's dictionary entry
    for flat_label, flat_data in example_data["lattice_of_flats"].items():
        file_content += f"        # Flat {flat_data['elements']}\n"
        file_content += f"        \"{flat_label}\": {{\n"
        file_content += f"            \"elements\": {flat_data['elements']},\n"
        file_content += f"            \"rank\": {flat_data['rank']},\n"
        file_content += f"            \"poincare_polynomial\": \"{flat_data['poincare_polynomial']}\",\n"
        file_content += f"        }},\n"

    file_content += f"    }},\n"

    # Write the content to the file
    filename = f"{filename_prefix}.py"
    with open(filename, "w") as f:
        f.write(file_content)

    print(f"\nSuccessfully created file: {filename}\n{'='*40}\n")


# --- Part 3: Main Execution ---

if __name__ == "__main__":
    # Define the list of matroids we want to generate examples for
    matroids_to_generate = {
        "u_2_3": matroids.Uniform(2, 3),
        "u_2_4": matroids.Uniform(2, 4),
        "fano": matroids.catalog.Fano(),
        "whirl_3": matroids.Whirl(3) # A non-representable matroid
    }

    # Loop through and generate a file for each one
    for prefix, matroid_obj in matroids_to_generate.items():
        create_example_file(matroid_obj, prefix)

    print("All example files have been generated.")



Processing U(2, 3): Matroid of rank 2 on 3 elements with circuit-closures
{2: {{0, 1, 2}}}...


AttributeError: 'sage.matroids.basis_matroid.BasisMatroid' object has no attribute 'characteristic_polynomial'